## Выгрузка данных через api devtodev

Документация - https://docs.devtodev.com/integration/server-api/raw-export

In [1]:
import requests
import datetime as dt
import time
import tempfile, os, zipfile
import pandas as pd

import os
from dotenv import load_dotenv

In [2]:
def raw_data(name, start, end, app_id):
    '''
    Выгрузка данных из devtodev

    name - название таблицы, которую нужно выгрузить
    start - дата начала выгрузки в формате unix
    end - дата конца выгрузки в формате unix
    app_id - идентификатор приложения
    '''

    load_dotenv()

    URL_DEVTODEV = os.getenv('URL_DEVTODEV')
    USER_TOKEN = os.getenv('USER_TOKEN')
    URL_STATUS_JOB = os.getenv('URL_STATUS_JOB')

    url = URL_DEVTODEV

    json_for_job = {"user_token": USER_TOKEN,  # It is obligatory, if it’s not sent with GET.
                    "app_id": app_id,  # Obligatory. Application identificator.
                    "start_date": start, # Obligatory. Unix time of export start date.
                    "end_date": end, # Obligatory. Unix time of export end date.
                    "events": name}

    # создаем задачу
    job = requests.post(url, json=json_for_job).json()

    url_for_status_job = URL_STATUS_JOB

    json_for_status_job = {"user_token": USER_TOKEN,
                           "job_id": "%s" % job['data']}

    status_job = {}

    status_job = requests.post(
        url_for_status_job, json=json_for_status_job).json()

    # проверяем статус задачи
    if status_job['status_code'] != 200 and status_job['status_code'] != 201:
        return print(job)

    while status_job['status_code'] == 200:
        time.sleep(20)
        status_job = requests.post(
            url_for_status_job, json=json_for_status_job).json()

    data_on_request = requests.get(status_job['data']['result']['url'])

    # распаковываем файл с данными в папку ../Data_from_devtodev
    file = tempfile.TemporaryFile()
    file.write(data_on_request.content)
    fzip = zipfile.ZipFile(file)
    fzip.extractall('../Data_from_devtodev')
    file.close()
    fzip.close()

    return


In [3]:
def unix_date(date,N=0):
    '''
    Перевод datetime в unix дату

    date - дата, которую нужно перевести
    N - количество дней, которые нужно прибавить к дате
    '''
    delta = dt.timedelta(days = N)
    x = date + delta
    unixtime = time.mktime(x.timetuple())

    return unixtime